In [4]:
import tensorflow as tf
import keras
from stable_diffusion.layers import PaddedConv2D
from stable_diffusion.autoencoder_kl import ResnetBlock, AttentionBlock
from classifier import GatedDense

In [5]:
def load_cifar_10():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    n_classes = 10
    X_train = (X_train / 127.5) -1
    X_test = (X_test / 127.5) -1
    y_train = tf.keras.utils.to_categorical(y_train, n_classes)
    y_test = tf.keras.utils.to_categorical(y_test, n_classes)
    return (X_train, y_train), (X_test, y_test)

In [6]:
class Classifier(keras.Sequential):
    def __init__(self):
        print("Encoder init")
        super().__init__(
            [
                PaddedConv2D(128, 3, padding=1 ),
                ResnetBlock(128,128),
                ResnetBlock(128, 128),
                PaddedConv2D(128 , 3 , padding=(0,1), stride=2),

                ResnetBlock(128,256),
                ResnetBlock(256, 256),
                PaddedConv2D(256 , 3 , padding=(0,1), stride=2),

                ResnetBlock(256,512),
                ResnetBlock(512, 512),
                PaddedConv2D(512, 3, padding=(0,1), stride=2),

                ResnetBlock(512,512),
                ResnetBlock(512, 512),

                ResnetBlock(512, 512),
                AttentionBlock(512),
                ResnetBlock(512, 512),

                keras.layers.GroupNormalization(epsilon=1e-5),
                keras.layers.Activation("swish"),
                PaddedConv2D(8, 3, padding=1 ),
                PaddedConv2D(8, 1 ),
                keras.layers.Lambda(lambda x : x[..., :4] * 0.18215),

                #############################

                keras.layers.Flatten(),
                GatedDense(32, activation=keras.activations.relu),
                keras.layers.Dropout(0.2),
                GatedDense(32, activation=None),
                keras.layers.Dense(10, activation='softmax')
            ]
        )

In [ ]:
(X_train, y_train), (X_test, y_test) = load_cifar_10()

model = Classifier()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()]
)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_test))

Encoder init
Epoch 1/100


2023-05-09 04:09:09.010935: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  5/391 [..............................] - ETA: 46:01 - loss: 2.3017 - categorical_accuracy: 0.1203